In [ ]:
''' Hyper parameter '''
LEARNING_RATE = 0.001
SAMPLE_LENGTH = 12
PREDICT_LENGTH = 6
BATCH_SIZE = 32

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import torch.utils.data as Data
import numpy as np

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
# input_size = 784 # 28x28
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

input_size = 28
sequence_length = 28
hidden_size = 128
num_layers = 2

In [ ]:
''' 數據預處理，將數據整理成 DataLoader 可讀取之格式 '''


pop_index = 7

import os
target_folder_path = '../data/event/Siou_Luan'
list_of_all_file = os.listdir(target_folder_path)

print(list_of_all_file)
test_seq_path = list_of_all_file.pop(pop_index)
print(list_of_all_file)

In [ ]:
print(len(list_of_all_file))

In [ ]:
sample_length = SAMPLE_LENGTH
predict_length = PREDICT_LENGTH

train_seq = []
train_seq_label_seq = []

for file_index in range(len(list_of_all_file)):
    print(list_of_all_file[file_index])
    target_df = pd.read_csv('../data/event/Siou_Luan_Ver2/'+str(list_of_all_file[file_index]))
    target_df['Time'] = pd.to_datetime(target_df['Time'])

    '''將降雨量/流量 標準化'''
    df_dimension = int(target_df.shape[0])
    Y = target_df[['rain_白石']]
    # scaler = MinMaxScaler(feature_range=(0, 1)).fit(Y)
    # Y = scaler.transform(Y)

    Z = target_df[['Flow']]
    # scaler = MinMaxScaler(feature_range=(0, 1)).fit(Z)
    # Z = scaler.transform(Z)

    X = target_df[['Time']]
    plt.plot(X, Y)
    # plt.plot(X, Z)
    plt.show()

    for i in range(len(Y)):
        if i+sample_length+predict_length < len(Y)+1:
            train_seq.append(Y[i:i+sample_length])
            train_seq_label_seq.append(Z[i+sample_length:i+sample_length+predict_length])

In [ ]:
train_seq_label_seq = np.array(train_seq_label_seq)
train_seq = np.array(train_seq)

In [ ]:
''' 設定 Dataset '''
train_seq = torch.tensor(train_seq)
train_seq_label_seq = torch.tensor(train_seq_label_seq)
torch_dataset = Data.TensorDataset(train_seq, train_seq_label_seq)

In [ ]:
''' 設定 DataLoader '''
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

dataloader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8
)

def show_batch():
    for epoch in range(1):
        for step, (batch_x, batch_y) in enumerate(dataloader):
            print("step:{}\nbatch_x:{}\nbatch_y:{}".format(step, batch_x, batch_y))
# x: 12 x 1
# y: 6 x 1
if __name__ == '__main__':
    show_batch()

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()

        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = self.input_size,
                            hidden_size = self.hidden_size,
                            num_layers = self.num_layers,
                            batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)

        # or:
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        # self.fc = nn.Linear(hidden_size, num_classes)
        

    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # x: (n, 28, 28), h0: (2, n, 128)

        # Forward propagate RNN
        out, _ = self.lstm(x, h0)
        # or:
        #out, _ = self.lstm(x, (h0,c0))

        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)

        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)

        out = self.fc(out)
        # out: (n, 10)
        return out

In [ ]:
model = LSTM()

In [ ]:
''' 模型訓練階段 '''
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)